In [1]:
import pandas as pd
import re
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
at_least_1_office = db.companies.find({"offices":{"$not":{"$size":0}}})


In [3]:
df = pd.DataFrame(at_least_1_office)

In [4]:
def getFirst(data):

    data = data['offices']

    principal = None

    if data[0]['latitude'] and data[0]['longitude']:

        principal = {

            "type":"Point",

            "coordinates":[data[0]['longitude'], data[0]['latitude']]

        }

    return {

        "totalOffices": len(data),

        "lat": data[0]['latitude'],

        "lng": data[0]['longitude'],

        "oficina_principal": principal

    }

Geoloc = df[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [5]:
df.drop(['_id', 'acquisition','created_at', 'crunchbase_url', 'ipo', 'homepage_url', 'image', 'partners', 'providerships', 'updated_at',  'permalink', 'products', 'external_links', 'acquisitions', 'alias_list', 'blog_feed_url', 'blog_url', 'category_code', 'deadpooled_day', 'deadpooled_month', 'deadpooled_url', 'email_address', 'founded_day', 'founded_month', 'phone_number', 'screenshots', 'tag_list', 'twitter_username', 'video_embeds', 'relationships', 'milestones', 'investments', 'overview', 'funding_rounds', 'competitions'] , axis=1, inplace=True)

In [6]:
df= df[['description', 'founded_year', 'name', 'total_money_raised', 'number_of_employees', 'deadpooled_year']].fillna('Unknown')

In [7]:
DataBase = pd.concat([df, Geoloc], axis=1)

In [8]:
def showmedamoney (m):
    
    m=str(m)
    
    floatmoney = float((re.findall('\d+|\d+.?\d+', m))[0])
    
    if m.startswith('C$'):x = floatmoney*1.31
    elif m.startswith('€'):x = floatmoney*0.89
    elif m.startswith('$'):x = floatmoney
    elif m.startswith('£'):x = floatmoney*0.80
    elif m.startswith('¥'):x = floatmoney*108.96
    else:
        x=floatmoney
        
    if m.endswith('M'):
        x = x*1000000
        return x
    elif m.endswith('B'):
        x = x*1000000000
        return x
    elif m.endswith('k'):
        x = x*1000
        return x
    else:
        return x

In [9]:
DataBase['total_money_raised']= DataBase['total_money_raised'].apply(showmedamoney)

In [10]:
def activity (m):
    m = str(m)
    
    if re.search("s*(design)|(Design)\s*", m):
        return m
    elif re.search("s*(Video)|(video)\s*", m):
        return m
    elif re.search("s*(Game)|(game)\s*", m):
        return m
    elif re.search("s*(Gaming)|(gaming)\s*", m):
        return m
    
    else:
        return 'OTHERS'



In [11]:
DataBase['description']= DataBase['description'].apply(activity)

In [12]:
DataBase=DataBase[DataBase['description']!='OTHERS']

In [13]:
DataBase['total_money_raised'] = DataBase['total_money_raised'].fillna(0.0).astype(int)

In [14]:
DataBase=DataBase[DataBase['deadpooled_year']=='Unknown']

In [15]:
DataBase

,description,founded_year,name,total_money_raised,number_of_employees,deadpooled_year,lat,lng,oficina_principal,totalOffices
33,Online & Mobile Video Platform,2006,Kyte,23000000,40,Unknown,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,...",1.0
46,Awesome Photo & Video Sharing,2002,SmugMug,0,62,Unknown,37.390056,-122.067692,"{'type': 'Point', 'coordinates': [-122.067692,...",1.0
51,Online video sharing,2005,YouTube,11000000,0,Unknown,37.627971,-122.426804,"{'type': 'Point', 'coordinates': [-122.426804,...",1.0
55,Live Social Video Community,2007,Ustream,60000000,250,Unknown,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ...",3.0
78,comedy video website,2007,Funny Or Die,18000000,Unknown,Unknown,37.421621,-122.137444,"{'type': 'Point', 'coordinates': [-122.137444,...",1.0
100,Video sharing service,2005,Dailymotion,68000000,120,Unknown,NaN,NaN,NaN,4.0
102,Social Gaming Company,2005,RockYou,136000000,106,Unknown,37.484619,-122.206893,"{'type': 'Point', 'coordinates': [-122.206893,...",1.0
118,Video guide for hotels,2007,TVtrip,13000000,Unknown,Unknown,48.856667,2.350987,"{'type': 'Point', 'coordinates': [2.350987, 48...",1.0
151,Video Processing Software,2005,MotionDSP,0,25,Unknown,NaN,NaN,NaN,1.0
189,video advertising network,2005,ScanScout,17000000,40,Unknown,42.340889,-71.051931,"{'type': 'Point', 'coordinates': [-71.051931, ...",3.0


In [16]:
DataBase.to_json('/home/javier/Documentos/visualizing-real-world-data-project/DataBase_py_clean.json', orient='records')